# IMDB 영화 리뷰 감성 분석 - Conv1D

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
import warnings
warnings.filterwarnings('ignore')

### Conv1D로 IMDB 리뷰 감성 분석
- 단어 빈도수: 10,000 (총 88,584)
- 문장의 단어수: 500 (최대 2,494)
- test data중 40%(10,000개)는 검증용으로

In [2]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
X_train.shape, X_test.shape, y_train.shape

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [6]:
from sklearn.model_selection import train_test_split
X_test, X_valid, y_test, y_valid = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.4, random_state=seed
)
X_test.shape, X_valid.shape, y_test.shape, y_valid.shape

((15000, 500), (10000, 500), (15000,), (10000,))

- Case 1) Conv1D x 2, MaxPooling1D x 2, Dropout, GlobalMaxPooling1D
    - embedding dim: 100

In [7]:
model1 = Sequential([ 
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [8]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [9]:
hist1 = model1.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
196/196 [==============================] - ETA: 0s - loss: 0.5400 - accuracy: 0.6996
Epoch 00001: val_loss improved from inf to 0.35259, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 15s 19ms/step - loss: 0.5400 - accuracy: 0.6996 - val_loss: 0.3526 - val_accuracy: 0.8431
Epoch 2/30
193/196 [============================>.] - ETA: 0s - loss: 0.2818 - accuracy: 0.8860
Epoch 00002: val_loss improved from 0.35259 to 0.32005, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 3s 15ms/step - loss: 0.2818 - accuracy: 0.8859 - val_loss: 0.3200 - val_accuracy: 0.8615
Epoch 3/30
193/196 [============================>.] - ETA: 0s - loss: 0.2071 - accuracy: 0.9220
Epoch 00003: val_loss did not improve from 0.32005
196/196 [==============================] - 3s 15ms/step - loss: 0.2071 - accuracy: 0.9220 - val_loss: 0.3343 - val_accuracy: 0.8607
Epoch 4/30
193/196 [============================>.] - ETA: 0s - loss: 0.1547 - accur

In [10]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 3ms/step - loss: 0.3159 - accuracy: 0.8651


[0.3159463405609131, 0.8651333451271057]

- Case 2) Conv1D + LSTM

In [11]:
from tensorflow.keras.layers import LSTM

In [12]:
model2 = Sequential([ 
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [13]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [14]:
hist2 = model2.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
195/196 [============================>.] - ETA: 0s - loss: 0.4300 - accuracy: 0.7854
Epoch 00001: val_loss improved from inf to 0.30058, saving model to best-imdb-conv1d-lstm.h5
196/196 [==============================] - 9s 27ms/step - loss: 0.4298 - accuracy: 0.7854 - val_loss: 0.3006 - val_accuracy: 0.8709
Epoch 2/30
193/196 [============================>.] - ETA: 0s - loss: 0.2156 - accuracy: 0.9191
Epoch 00002: val_loss improved from 0.30058 to 0.28859, saving model to best-imdb-conv1d-lstm.h5
196/196 [==============================] - 5s 23ms/step - loss: 0.2162 - accuracy: 0.9189 - val_loss: 0.2886 - val_accuracy: 0.8803
Epoch 3/30
196/196 [==============================] - ETA: 0s - loss: 0.1566 - accuracy: 0.9433
Epoch 00003: val_loss did not improve from 0.28859
196/196 [==============================] - 4s 23ms/step - loss: 0.1566 - accuracy: 0.9433 - val_loss: 0.3202 - val_accuracy: 0.8825
Epoch 4/30
194/196 [============================>.] - ETA: 0s - loss: 0.121

In [15]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 3s 5ms/step - loss: 0.2876 - accuracy: 0.8809


[0.28758394718170166, 0.8808666467666626]

- Case 3) Conv1D + Dense

In [18]:
model3 = Sequential([ 
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),  
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_4 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 100)              

In [20]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-fcn.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [21]:
hist3 = model3.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
192/196 [============================>.] - ETA: 0s - loss: 0.5734 - accuracy: 0.6825
Epoch 00001: val_loss improved from inf to 0.35592, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 5s 19ms/step - loss: 0.5710 - accuracy: 0.6843 - val_loss: 0.3559 - val_accuracy: 0.8430
Epoch 2/30
195/196 [============================>.] - ETA: 0s - loss: 0.2959 - accuracy: 0.8772
Epoch 00002: val_loss improved from 0.35592 to 0.27714, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 4s 18ms/step - loss: 0.2959 - accuracy: 0.8772 - val_loss: 0.2771 - val_accuracy: 0.8827
Epoch 3/30
194/196 [============================>.] - ETA: 0s - loss: 0.2025 - accuracy: 0.9227
Epoch 00003: val_loss improved from 0.27714 to 0.26338, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 3s 14ms/step - loss: 0.2027 - accuracy: 0.9226 - val_loss: 0.2634 - val_accuracy: 0.8925
Epoch 4/30
193/196 [============

In [22]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 3ms/step - loss: 0.2667 - accuracy: 0.8919


[0.26671740412712097, 0.8918666839599609]